In [59]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from collections import Counter

In [60]:
# 한글 깨짐 방지 코드
import matplotlib
from matplotlib import font_manager, rc
import platform
if platform.system()=="Windows":
    font_name=font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
matplotlib.rcParams['axes.unicode_minus']=False

import warnings
warnings.filterwarnings("ignore")

In [61]:
region_pop = pd.read_csv('./data/report.txt', sep='\t', header=2)
region_pop.head()

,기간,자치구,세대,계,남자,여자,계.1,남자.1,여자.1,계.2,남자.2,여자.2,세대당인구,65세이상고령자
0,2020.3/4,합계,"4,405,833","9,953,009","4,840,912","5,112,097","9,699,232","4,719,170","4,980,062","253,777","121,742","132,035",2.20,"1,552,356"
1,2020.3/4,종로구,"74,861","159,842","77,391","82,451","149,952","73,024","76,928","9,890","4,367","5,523",2.00,"28,396"
2,2020.3/4,중구,"63,594","135,321","66,193","69,128","125,800","61,526","64,274","9,521","4,667","4,854",1.98,"24,265"
3,2020.3/4,용산구,"112,451","244,953","119,074","125,879","229,786","110,604","119,182","15,167","8,470","6,697",2.04,"39,995"
4,2020.3/4,성동구,"136,096","302,695","147,582","155,113","295,591","144,444","151,147","7,104","3,138","3,966",2.17,"45,372"


In [62]:
region_pop['자치구']

region_pop = region_pop[region_pop['자치구']!='합계']

In [63]:
region_pop.head(3)

,기간,자치구,세대,계,남자,여자,계.1,남자.1,여자.1,계.2,남자.2,여자.2,세대당인구,65세이상고령자
1,2020.3/4,종로구,"74,861","159,842","77,391","82,451","149,952","73,024","76,928","9,890","4,367","5,523",2.00,"28,396"
2,2020.3/4,중구,"63,594","135,321","66,193","69,128","125,800","61,526","64,274","9,521","4,667","4,854",1.98,"24,265"
3,2020.3/4,용산구,"112,451","244,953","119,074","125,879","229,786","110,604","119,182","15,167","8,470","6,697",2.04,"39,995"


In [64]:
business = pd.read_csv('./data/report2.txt', sep='\t', header=2)
business.head(3)

,기간,자치구,동,사업체수,여성대표자,계,남,여,사업체수.1,종사자수,...,사업체수.15,종사자수.14,사업체수.16,종사자수.15,사업체수.17,종사자수.16,사업체수.18,종사자수.17,사업체수.19,종사자수.18
0,2018,합계,합계,"823,385","279,711","5,210,936","2,878,227","2,332,709",30,462,...,"1,282","140,121","35,377","346,219","28,824","378,366","23,025","91,104","69,694","191,423"
1,2018,종로구,소계,"39,952","13,442","265,017","152,717","112,300",3,17,...,89,"15,658",850,"13,504",744,"15,888",909,"6,227","2,448","7,672"
2,2018,종로구,사직동,"3,541","1,191","49,536","28,658","20,878",1,14,...,19,"6,312",86,967,103,870,95,"1,544",252,843


In [65]:
business['자치구']

business = business[business['자치구']!='합계']

In [66]:
business.head(2)

,기간,자치구,동,사업체수,여성대표자,계,남,여,사업체수.1,종사자수,...,사업체수.15,종사자수.14,사업체수.16,종사자수.15,사업체수.17,종사자수.16,사업체수.18,종사자수.17,사업체수.19,종사자수.18
1,2018,종로구,소계,"39,952","13,442","265,017","152,717","112,300",3,17,...,89,"15,658",850,"13,504",744,"15,888",909,"6,227","2,448","7,672"
2,2018,종로구,사직동,"3,541","1,191","49,536","28,658","20,878",1,14,...,19,"6,312",86,967,103,870,95,"1,544",252,843


In [67]:
seoul_star = pd.read_excel('seoul_starbucks_list.xlsx')
seoul_star

,매장명,위도,경도,타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,리저브 매장 2번,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,리저브 매장 2번,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.514132,127.020563,리저브 매장 2번,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,리저브 매장 2번,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,스탈릿대치R,37.494668,127.062583,리저브 매장 2번,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232
...,...,...,...,...,...,...
551,사가정역,37.579594,127.087966,리저브 매장 2번,서울특별시 중랑구 면목로 310,1522-3232
552,상봉역,37.596890,127.086470,리저브 매장 2번,서울특별시 중랑구 망우로 307 (상봉동),1522-3232
553,묵동이마트,37.613433,127.077484,리저브 매장 2번,"서울특별시 중랑구 동일로 932 (묵동, 묵동자이아파트) (묵동이마트 B1층)",1522-3232
554,묵동,37.615368,127.076633,리저브 매장 2번,서울특별시 중랑구 동일로 952,1522-3232


In [68]:
county_pop = pd.read_excel('sgg_pop.xlsx')
county_pop.head()

,시군구명,주민등록인구
0,종로구,"159,842"
1,중구,"135,321"
2,용산구,"244,953"
3,성동구,"302,695"
4,광진구,"361,923"


In [69]:
county_pop.sort_values(by='시군구명', inplace=True)

In [70]:
county_pop.head()

,시군구명,주민등록인구
22,강남구,"544,085"
24,강동구,"462,275"
8,강북구,"313,550"
15,강서구,"589,536"
20,관악구,"514,555"


---

In [71]:
'''
*판다스를 사용하지 않고 파이썬 기반으로 하시면 됨. (판다스 사용하셔도 좋음)
1. 
- 구 단위 스타벅스 매장 개수?
- 구 단위 인구수?
- 각 구 단위로 스타벅스 매장 밀도(스타벅스 1개 당 인구수 비율)
- 가장 높은(낮은) 밀도 5개 구 => 스타벅스 입점 위치 정할때 활용?

2.
- 구 단위 사업체 수(근로자의 수)
- 각 구 단위로 스타벅스 매장 밀도(스타벅스 1개 당 사업체/근로자 수 비율)
- 가장 높은(낮은) 밀도 5개 구 => 스타벅스 입점 위치 정할때 활용?

3.
1, 2번 문제를 종합 분석 => 인구가 많다고 해서 스타벅스 이용률이 높을까?
근로자 수가 많다고 해서 스타벅스 이용률이 높을까?
인구와 근로자수 중에서 스타벅스 이용률에 영향 크게 미치는 요인이 뭘까?
'''

'\n*판다스를 사용하지 않고 파이썬 기반으로 하시면 됨. (판다스 사용하셔도 좋음)\n1. \n- 구 단위 스타벅스 매장 개수?\n- 구 단위 인구수?\n- 각 구 단위로 스타벅스 매장 밀도(스타벅스 1개 당 인구수 비율)\n- 가장 높은(낮은) 밀도 5개 구 => 스타벅스 입점 위치 정할때 활용?\n\n2.\n- 구 단위 사업체 수(근로자의 수)\n- 각 구 단위로 스타벅스 매장 밀도(스타벅스 1개 당 사업체/근로자 수 비율)\n- 가장 높은(낮은) 밀도 5개 구 => 스타벅스 입점 위치 정할때 활용?\n\n3.\n1, 2번 문제를 종합 분석 => 인구가 많다고 해서 스타벅스 이용률이 높을까?\n근로자 수가 많다고 해서 스타벅스 이용률이 높을까?\n인구와 근로자수 중에서 스타벅스 이용률에 영향 크게 미치는 요인이 뭘까?\n'

In [72]:
seoul_star_gu1=[]
for i in range(len(seoul_star['주소'])):
    seoul_star_gu1.append(re.findall('\w{2,3}구|중구', seoul_star['주소'][i]))

seoul_star_gu=[]
for i in range(len(seoul_star['주소'])):
    seoul_star_gu.append(seoul_star_gu1[i][0])

In [73]:
county_pop['시군구명']

22     강남구
24     강동구
8      강북구
15     강서구
20     관악구
4      광진구
16     구로구
17     금천구
10     노원구
9      도봉구
5     동대문구
19     동작구
13     마포구
12    서대문구
21     서초구
3      성동구
7      성북구
23     송파구
14     양천구
18    영등포구
2      용산구
11     은평구
0      종로구
1       중구
6      중랑구
Name: 시군구명, dtype: object

In [74]:
array1 = np.array(seoul_star_gu)

In [75]:
columns=['구']
gu_df = pd.DataFrame(array1, columns=columns)

In [76]:
gu_df

,구
0,강남구
1,강남구
2,강남구
3,강남구
4,강남구
...,...
551,중랑구
552,중랑구
553,중랑구
554,중랑구


In [77]:
final_seoul_star = pd.concat([seoul_star, gu_df], axis=1, ignore_index=True)

In [78]:
columns={0: '지점명',1: '위도',2: '경도',3: '매장코드',4: '주소',5: '전화번호',6: '시군구명'}

In [79]:
final_seoul_star.rename(columns=columns, inplace=True)

In [80]:
final_seoul_star

,지점명,위도,경도,매장코드,주소,전화번호,시군구명
0,역삼아레나빌딩,37.501087,127.043069,리저브 매장 2번,서울특별시 강남구 언주로 425 (역삼동),1522-3232,강남구
1,논현역사거리,37.510178,127.022223,리저브 매장 2번,서울특별시 강남구 강남대로 538 (논현동),1522-3232,강남구
2,신사역성일빌딩,37.514132,127.020563,리저브 매장 2번,서울특별시 강남구 강남대로 584 (논현동),1522-3232,강남구
3,국기원사거리,37.499517,127.031495,리저브 매장 2번,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,강남구
4,스탈릿대치R,37.494668,127.062583,리저브 매장 2번,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,강남구
...,...,...,...,...,...,...,...
551,사가정역,37.579594,127.087966,리저브 매장 2번,서울특별시 중랑구 면목로 310,1522-3232,중랑구
552,상봉역,37.596890,127.086470,리저브 매장 2번,서울특별시 중랑구 망우로 307 (상봉동),1522-3232,중랑구
553,묵동이마트,37.613433,127.077484,리저브 매장 2번,"서울특별시 중랑구 동일로 932 (묵동, 묵동자이아파트) (묵동이마트 B1층)",1522-3232,중랑구
554,묵동,37.615368,127.076633,리저브 매장 2번,서울특별시 중랑구 동일로 952,1522-3232,중랑구


---

In [81]:
starbucks_gu = final_seoul_star.groupby('시군구명')['지점명'].count()
starbucks_gu = starbucks_gu.reset_index()
starbucks_gu = starbucks_gu.set_index('시군구명')
starbucks_gu

,지점명
시군구명,
강남구,85
강동구,15
강북구,5
강서구,19
관악구,11
광진구,17
구로구,11
금천구,11
노원구,13


In [82]:
starbucks_gu.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 강남구 to 중랑구
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   지점명     25 non-null     int64
dtypes: int64(1)
memory usage: 400.0+ bytes


In [83]:
merge_inner = pd.merge(starbucks_gu, county_pop, on='시군구명')

In [84]:
merge_inner

,시군구명,지점명,주민등록인구
0,강남구,85,"544,085"
1,강동구,15,"462,275"
2,강북구,5,"313,550"
3,강서구,19,"589,536"
4,관악구,11,"514,555"
5,광진구,17,"361,923"
6,구로구,11,"435,751"
7,금천구,11,"249,641"
8,노원구,13,"529,532"
9,도봉구,3,"329,300"


In [85]:
len(merge_inner)

25

In [86]:
density_pop=[]
for i in range(len(merge_inner)):
    population = int(merge_inner['주민등록인구'][i].replace(",", "")) // int(merge_inner['지점명'][i])
    density_pop.append(population)

In [87]:
density_pop

[6401,
 30818,
 62710,
 31028,
 46777,
 21289,
 39613,
 22694,
 40733,
 109766,
 39853,
 36783,
 11984,
 16193,
 8958,
 27517,
 32133,
 20505,
 27061,
 10683,
 11664,
 60730,
 3996,
 2416,
 50123]

In [88]:
density_pop1 = np.array(density_pop)

density_pop = pd.DataFrame(density_pop1, columns=['인구수대비밀도'])

In [89]:
density_pop

,인구수대비밀도
0,6401
1,30818
2,62710
3,31028
4,46777
5,21289
6,39613
7,22694
8,40733
9,109766


In [90]:
columns={0: '시군구명',1: '지점명',2: '주민등록인구',3: '인구수대비밀도'}

In [91]:
part_of_star = pd.concat([merge_inner, density_pop], axis=1, ignore_index=True)

In [92]:
part_of_star.rename(columns=columns, inplace=True)

In [93]:
part_of_star.head()

,시군구명,지점명,주민등록인구,인구수대비밀도
0,강남구,85,"544,085",6401
1,강동구,15,"462,275",30818
2,강북구,5,"313,550",62710
3,강서구,19,"589,536",31028
4,관악구,11,"514,555",46777


In [94]:
part_of_star = part_of_star.sort_values('인구수대비밀도', ascending=False)

In [95]:
part_of_star[['시군구명', '인구수대비밀도']][:5]

,시군구명,인구수대비밀도
9,도봉구,109766
2,강북구,62710
21,은평구,60730
24,중랑구,50123
4,관악구,46777


In [96]:
part_of_star[['시군구명', '인구수대비밀도']][-5:]

,시군구명,인구수대비밀도
19,영등포구,10683
14,서초구,8958
0,강남구,6401
22,종로구,3996
23,중구,2416


---

In [97]:
business.head()

,기간,자치구,동,사업체수,여성대표자,계,남,여,사업체수.1,종사자수,...,사업체수.15,종사자수.14,사업체수.16,종사자수.15,사업체수.17,종사자수.16,사업체수.18,종사자수.17,사업체수.19,종사자수.18
1,2018,종로구,소계,"39,952","13,442","265,017","152,717","112,300",3,17,...,89,"15,658",850,"13,504",744,"15,888",909,"6,227","2,448","7,672"
2,2018,종로구,사직동,"3,541","1,191","49,536","28,658","20,878",1,14,...,19,"6,312",86,967,103,870,95,"1,544",252,843
3,2018,종로구,삼청동,712,324,"4,577","2,193","2,384",-,-,...,4,"1,204",14,260,7,30,17,175,24,209
4,2018,종로구,부암동,565,240,"3,609","1,706","1,903",-,-,...,3,59,40,"1,424",15,148,16,40,83,362
5,2018,종로구,평창동,751,319,"3,565","1,670","1,895",-,-,...,3,85,56,618,36,466,32,186,115,262


In [98]:
gu = business['자치구']
gu = gu.reset_index()

In [99]:
business[['자치구','사업체수']]

,자치구,사업체수
1,종로구,"39,952"
2,종로구,"3,541"
3,종로구,712
4,종로구,565
5,종로구,751
...,...,...
445,강동구,19
446,강동구,"1,601"
447,강동구,"1,916"
448,강동구,"3,736"


In [100]:
cor_pop=[]
for i in range(1, 450):
    cor_pop.append(int(business['사업체수'][i].replace(',', '')))

In [101]:
cor_pop= np.array(cor_pop)

In [102]:
cor_pop = pd.DataFrame(cor_pop, columns=['사업체수'])

In [103]:
cor_pop

,사업체수
0,39952
1,3541
2,712
3,565
4,751
...,...
444,19
445,1601
446,1916
447,3736


In [104]:
business_pop = pd.concat([gu, cor_pop], axis=1, ignore_index=True)

In [105]:
business_pop

,0,1,2
0,1,종로구,39952
1,2,종로구,3541
2,3,종로구,712
3,4,종로구,565
4,5,종로구,751
...,...,...,...
444,445,강동구,19
445,446,강동구,1601
446,447,강동구,1916
447,448,강동구,3736


In [106]:
columns = { 1: '자치구', 2: '사업체수'}

In [107]:
business_pop = business_pop.drop([0], axis=1)



In [108]:
business_pop

,1,2
0,종로구,39952
1,종로구,3541
2,종로구,712
3,종로구,565
4,종로구,751
...,...,...
444,강동구,19
445,강동구,1601
446,강동구,1916
447,강동구,3736


In [109]:
business_pop.rename(columns=columns, inplace=True)

In [110]:
business_pop

,자치구,사업체수
0,종로구,39952
1,종로구,3541
2,종로구,712
3,종로구,565
4,종로구,751
...,...,...
444,강동구,19
445,강동구,1601
446,강동구,1916
447,강동구,3736


In [111]:
business_sum = business_pop.groupby('자치구').sum()
business_sum

,사업체수
자치구,
강남구,142746
강동구,59544
강북구,37836
강서구,77048
관악구,52210
광진구,49070
구로구,77106
금천구,64944
노원구,53232


In [112]:
business_sum.sort_values(by='사업체수', ascending=False).head()

,사업체수
자치구,
강남구,142746
중구,121914
송파구,95544
서초구,93080
영등포구,85178


In [113]:
business_sum.sort_values(by='사업체수', ascending=False).tail()

,사업체수
자치구,
용산구,41626
동작구,39832
서대문구,39592
강북구,37836
도봉구,37788


---